# Process Results

## Imports

In [1]:
import pandas as pd
import cufflinks as cf
import pickle as pk
import sys
import numpy as np

sys.path.append('./..')

cf.go_offline()

In [2]:
from auxiliar.VectorizerHelper import vectorizer, vectorizerIdf, preprocessor
from auxiliar import parameters
from auxiliar.HtmlParser import HtmlParser

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import recall_score

In [4]:
colors=['red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue']

In [5]:
def compute_metrics(predictions, real):
    metrics = dict()
    bin_preds = predictions
    metrics['mse'] = mean_squared_error(bin_preds, real)
    metrics['recall'] = recall_score(bin_preds, real, average='micro')
    metrics['f1'] = f1_score(bin_preds, real, average='micro')
    metrics['acc'] = accuracy_score(bin_preds, real)
    return metrics

## Machine Learning

In [6]:
results_base_line = pd.read_pickle('machine_learning/tweeter/base_line/3-clases/results.pkl').to_dict()
results_grid = pd.read_pickle('machine_learning/tweeter/grid_search/3-clases/results-idf.pkl').to_dict()
results_test = pd.read_pickle('machine_learning/tweeter/grid_search/3-clases/test_results.pkl').to_dict()

In [7]:
with open('machine_learning/tweeter/grid_search/3-clases/grid_results-idf.pkl', 'rb') as fp:
    grid = pk.load(fp)

In [8]:
def get_results_df(res, nrange):
    keys = res.keys()
    results = []
    for k in keys:
        for i in range(nrange):
            results.append(compute_metrics(res[k]['predicted'][i], res[k]['real'][i]))
    results_df = pd.DataFrame(results).transpose()
    results_df.columns = pd.MultiIndex.from_product([keys, range(nrange)])
    results_df = results_df.transpose().reset_index().groupby(['level_0']).mean()
    results_df = results_df.drop(columns=['level_1'])
    return results_df

base line results over test set

In [9]:
def get_results_df_cine(res, nrange):
    keys = res.keys()
    results = []
    for k in keys:
        for i in range(nrange):
            results.append(compute_metrics(res[k]['cine_predicted'][i], res[k]['cine_real'][i]))
    results_df = pd.DataFrame(results).transpose()
    results_df.columns = pd.MultiIndex.from_product([keys, range(nrange)])
    results_df = results_df.transpose().reset_index().groupby(['level_0']).mean()
    results_df = results_df.drop(columns=['level_1'])
    return results_df

In [10]:
def get_results_df_2(res, nrange):
    keys = res.keys()
    results = []
    for k in keys:
        for i in range(nrange):
            results.append(compute_metrics(res[k]['predicted'][i], res[k]['real'][i]))
    results_df = pd.DataFrame(results).transpose()
    results_df.columns = pd.MultiIndex.from_product([keys, range(nrange)])
#     results_df = results_df.transpose().reset_index().groupby(['level_0']).mean()
#     results_df = results_df.drop(columns=['level_1'])
    return results_df

### Base line

In [15]:
get_results_df(results_base_line, 10).style.highlight_max()

,acc,f1,mse,recall
level_0,,,,
lr,0.729844,0.729844,0.905377,0.729844
ls,0.724713,0.724713,0.929319,0.724713
mb,0.740096,0.740096,0.80196,0.740096
rf,0.67712,0.67712,1.11627,0.67712


In [13]:
print(round(get_results_df(results_base_line, 10) * 100, 2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &    acc &     f1 &     mse &  recall \\
level\_0 &        &        &         &         \\
\midrule
lr      &  72.98 &  72.98 &   90.54 &   72.98 \\
ls      &  72.47 &  72.47 &   92.93 &   72.47 \\
mb      &  74.01 &  74.01 &   80.20 &   74.01 \\
rf      &  67.71 &  67.71 &  111.63 &   67.71 \\
\bottomrule
\end{tabular}



In [14]:
print(round(get_results_df_cine(results_base_line, 10) * 100, 2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &    acc &     f1 &     mse &  recall \\
level\_0 &        &        &         &         \\
\midrule
lr      &  31.19 &  31.19 &  150.51 &   31.19 \\
ls      &  28.63 &  28.63 &  162.97 &   28.63 \\
mb      &  37.35 &  37.35 &  128.21 &   37.35 \\
rf      &  32.78 &  32.78 &  137.66 &   32.78 \\
\bottomrule
\end{tabular}



means of grid search results over k-fold

### Grid search - train

In [46]:
get_results_df_2(results_grid, 10).stack().transpose()['f1'].transpose().iplot()

In [34]:
get_results_df(results_grid, 10).style.highlight_max()

,acc,f1,mse,recall
level_0,,,,
lr,0.736778,0.736778,0.882371,0.736778
ls,0.749544,0.749544,0.831307,0.749544
mb,0.746809,0.746809,0.842249,0.746809
rf,0.688146,0.688146,1.06778,0.688146


In [25]:
print(round(get_results_df(results_grid, 10) * 100, 2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &    acc &     f1 &     mse &  recall \\
level\_0 &        &        &         &         \\
\midrule
lr      &  73.68 &  73.68 &   88.24 &   73.68 \\
ls      &  74.95 &  74.95 &   83.13 &   74.95 \\
mb      &  74.68 &  74.68 &   84.22 &   74.68 \\
rf      &  68.81 &  68.81 &  106.78 &   68.81 \\
\bottomrule
\end{tabular}



Grid search results over test set

In [35]:
get_results_df(results_test, 1).style.highlight_max()

,acc,f1,mse,recall
level_0,,,,
lr,0.74318,0.74318,0.803726,0.74318
ls,0.752495,0.752495,0.766467,0.752495
mb,0.718563,0.718563,0.902196,0.718563
rf,0.696607,0.696607,0.988024,0.696607


In [36]:
print(round(get_results_df(results_test, 1) * 100, 2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &    acc &     f1 &    mse &  recall \\
level\_0 &        &        &        &         \\
\midrule
lr      &  74.32 &  74.32 &  80.37 &   74.32 \\
ls      &  75.25 &  75.25 &  76.65 &   75.25 \\
mb      &  71.86 &  71.86 &  90.22 &   71.86 \\
rf      &  69.66 &  69.66 &  98.80 &   69.66 \\
\bottomrule
\end{tabular}



In [13]:
print(round(get_results_df_cine(results_test, 1) * 100, 2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &    acc &     f1 &     mse &  recall \\
level\_0 &        &        &         &         \\
\midrule
lr      &  39.74 &  39.74 &  118.28 &   39.74 \\
ls      &  38.36 &  38.36 &  124.34 &   38.36 \\
mb      &  36.06 &  36.06 &  133.36 &   36.06 \\
rf      &  34.94 &  34.94 &  119.24 &   34.94 \\
\bottomrule
\end{tabular}



### Grid search - best params

In [63]:
grid['lr'].best_params_

{'lr__max_iter': 100,
 'prep__negation': 1,
 'prep__repeated_letters': True,
 'vect__tokenizer': <function auxiliar.VectorizerHelper.stemize(text)>}

In [64]:
grid['ls'].best_params_

{'ls__base_estimator__max_iter': 1000,
 'prep__negation': 1,
 'prep__repeated_letters': True,
 'vect__tokenizer': <function auxiliar.VectorizerHelper.stemize(text)>}

In [65]:
grid['mb'].best_params_

{'mb__alpha': 0.5,
 'prep__negation': 1,
 'prep__repeated_letters': True,
 'vect__tokenizer': <function auxiliar.VectorizerHelper.stemize(text)>}

In [66]:
grid['rf'].best_params_

{'prep__negation': 1,
 'prep__repeated_letters': True,
 'rf__n_estimators': 100,
 'vect__tokenizer': <function auxiliar.VectorizerHelper.stemize(text)>}

### Grid search - test

In [18]:
pd.DataFrame(results_test['lr']['real'][0]).iplot(kind='histogram')

In [19]:
pd.DataFrame([results_test['lr']['predicted'][0], results_test['ls']['predicted'][0],\
             results_test['mb']['predicted'][0], results_test['rf']['predicted'][0]], index=['lr', 'ls' , 'mb', 'rf'])\
.transpose().iplot(kind='histogram')

We could highlight Naive bayes model over the others because it seems to have better generalization over test cases

## Deep Learning

### Get Results

In [11]:
lstm_base = pd.read_pickle('deep_learning/tweeter/3-clases/lstm_val_lstm.pkl')
lstm_base_evas = pd.read_pickle('deep_learning/tweeter/3-clases/lstm_val_lstm_evas.pkl')
lstm_base_pred = pd.read_pickle('deep_learning/tweeter/3-clases/lstm_val_preds.pkl')
lstm_base_pred_cine = pd.read_pickle('deep_learning/tweeter/3-clases/lstm_valcine_preds.pkl')

lstm_simpler = pd.read_pickle('deep_learning/tweeter/3-clases/lstm_simple_lstm.pkl')
lstm_simpler_evas = pd.read_pickle('deep_learning/tweeter/3-clases/lstm_simple_lstm_evas.pkl')
lstm_simpler_preds = pd.read_pickle('deep_learning/tweeter/3-clases/lstm_simple_preds.pkl')
lstm_simpler_preds_cine = pd.read_pickle('deep_learning/tweeter/3-clases/lstm_simplecine_preds.pkl')

lstm_dropout = pd.read_pickle('deep_learning/tweeter/3-clases/dropout_lstm_lstm.pkl')
lstm_dropout_evas = pd.read_pickle('deep_learning/tweeter/3-clases/dropout_lstm_lstm_evas.pkl')
lstm_dropout_preds = pd.read_pickle('deep_learning/tweeter/3-clases/dropout_lstm_preds.pkl')
lstm_dropout_preds_cine = pd.read_pickle('deep_learning/tweeter/3-clases/dropout_lstmcine_preds.pkl')

lstm_dropout2 = pd.read_pickle('deep_learning/tweeter/3-clases/dropout2_lstm_lstm.pkl')
lstm_dropout2_evas = pd.read_pickle('deep_learning/tweeter/3-clases/dropout2_lstm_lstm_evas.pkl')
lstm_dropout2_preds = pd.read_pickle('deep_learning/tweeter/3-clases/dropout2_lstm_preds.pkl')
lstm_dropout2_preds_cine = pd.read_pickle('deep_learning/tweeter/3-clases/dropout2_lstmcine_preds.pkl')

lstm_bn = pd.read_pickle('deep_learning/tweeter/3-clases/bn_lstm_lstm.pkl')
lstm_bn_evas = pd.read_pickle('deep_learning/tweeter/3-clases/bn_lstm_lstm_evas.pkl')
lstm_bn_preds = pd.read_pickle('deep_learning/tweeter/3-clases/bn_lstm_preds.pkl')
lstm_bn_preds_cine = pd.read_pickle('deep_learning/tweeter/3-clases/bn_lstmcine_preds.pkl')

lstm_glorot = pd.read_pickle('deep_learning/tweeter/3-clases/glorot_lstm_lstm.pkl')
lstm_glorot_evas = pd.read_pickle('deep_learning/tweeter/3-clases/glorot_lstm_lstm_evas.pkl')
lstm_glorot_preds = pd.read_pickle('deep_learning/tweeter/3-clases/glorot_lstm_preds.pkl')
lstm_glorot_preds_cine = pd.read_pickle('deep_learning/tweeter/3-clases/glorot_lstmcine_preds.pkl')

lstm_glorot_wo_bn = pd.read_pickle('deep_learning/tweeter/3-clases/glorot__wobn_lstm_lstm.pkl')
lstm_glorot_wo_bn_evas = pd.read_pickle('deep_learning/tweeter/3-clases/glorot__wobn_lstm_lstm_evas.pkl')
lstm_glorot_wo_bn_preds = pd.read_pickle('deep_learning/tweeter/3-clases/glorot__wobn_lstm_preds.pkl')
lstm_glorot_wo_bn_preds_cine = pd.read_pickle('deep_learning/tweeter/3-clases/glorot__wobn_lstmcine_preds.pkl')


lstm_double = pd.read_pickle('deep_learning/tweeter/3-clases/double_lstm_lstm.pkl')
lstm_double_evas = pd.read_pickle('deep_learning/tweeter/3-clases/double_lstm_lstm_evas.pkl')
lstm_double_preds = pd.read_pickle('deep_learning/tweeter/3-clases/double_lstm_preds.pkl')
lstm_double_preds_cine = pd.read_pickle('deep_learning/tweeter/3-clases/double_lstmcine_preds.pkl')

lstm_conv = pd.read_pickle('deep_learning/tweeter/3-clases/convolutional_lstm.pkl')
lstm_conv_evas = pd.read_pickle('deep_learning/tweeter/3-clases/convolutional_lstm_evas.pkl')
lstm_conv_preds = pd.read_pickle('deep_learning/tweeter/3-clases/convolutional_preds.pkl')
lstm_conv_preds_cine = pd.read_pickle('deep_learning/tweeter/3-clases/convolutionalcine_preds.pkl')

lstm_conv1d = pd.read_pickle('deep_learning/tweeter/3-clases/convolutional1d_lstm.pkl')
lstm_conv1d_evas = pd.read_pickle('deep_learning/tweeter/3-clases/convolutional1d_lstm_evas.pkl')
lstm_conv1d_preds = pd.read_pickle('deep_learning/tweeter/3-clases/convolutional1d_preds.pkl')
lstm_conv1d_preds_cine = pd.read_pickle('deep_learning/tweeter/3-clases/convolutional1dcine_preds.pkl')

lstm_bidirectional = pd.read_pickle('deep_learning/tweeter/3-clases/bidirectional_lstm.pkl')
lstm_bidirectional_evas = pd.read_pickle('deep_learning/tweeter/3-clases/bidirectional_lstm_evas.pkl')
lstm_bidirectional_preds = pd.read_pickle('deep_learning/tweeter/3-clases/bidirectional_preds.pkl')
lstm_bidirectional_preds_cine = pd.read_pickle('deep_learning/tweeter/3-clases/bidirectionalcine_preds.pkl')


### Plot results

In [48]:
print('base')
lstm_base.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
print('simplification')
lstm_simpler.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
print('dropout')
lstm_dropout.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
print('dropout 0.2')
lstm_dropout2.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
print('batch normalization')
lstm_bn.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
print('glorot initialization')
lstm_glorot.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
print('glorot initialization without batch normalization')
lstm_glorot_wo_bn.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
print('double lstm')
lstm_double.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
print('convolutional lstm')
lstm_conv.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
print('convolutional 2d lstm')
lstm_conv1d.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
print('bidirectional lstm')
lstm_bidirectional.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)

base


simplification


dropout


dropout 0.2


batch normalization


...

glorot initialization without batch normalization


double lstm


convolutional lstm


convolutional 2d lstm


bidirectional lstm


In [49]:
print('base')
lstm_base_evas.iplot()
print('simplification')
lstm_simpler_evas.iplot()
print('dropout')
lstm_dropout_evas.iplot()
print('dropout 0.2')
lstm_dropout2_evas.iplot()
print('batch normalization')
lstm_bn_evas.iplot()
print('glorot initialization')
lstm_glorot_evas.iplot()
print('glorot initialization without batch normalization')
lstm_glorot_wo_bn_evas.iplot()
print('double lstm')
lstm_double_evas.iplot()
print('convolutional lstm')
lstm_conv_evas.iplot()
print('convolutional 2d lstm')
lstm_conv1d_evas.iplot()
print('bidirectional lstm')
lstm_bidirectional_evas.iplot()

base


simplification


dropout


dropout 0.2


batch normalization


...

glorot initialization without batch normalization


double lstm


convolutional lstm


convolutional 2d lstm


bidirectional lstm


### Table results

#### Train results

In [50]:
means = pd.concat([\
           lstm_base.stack(level=0).mean(),\
           lstm_simpler.stack(level=0).mean(),\
           lstm_dropout.stack(level=0).mean(),\
           lstm_dropout2.stack(level=0).mean(),\
           lstm_bn.stack(level=0).mean(),\
           lstm_glorot.stack(level=0).mean(),\
           lstm_glorot_wo_bn.stack(level=0).mean(),\
           lstm_double.stack(level=0).mean(),\
           lstm_conv.stack(level=0).mean(),\
           lstm_conv1d.stack(level=0).mean(),\
           lstm_bidirectional.stack(level=0).mean()
          ], axis=1)

In [51]:
means.columns = ['base', 'simpler', 'dropout', 'dropout 0.2', 'batch norm', 'glorot', 'glorot_wo_bn', 'double', 'conv', 'conv1d', 'bidirectional']

In [52]:
means.transpose().style.highlight_min(subset=pd.IndexSlice[:, ['val_loss']])

,acc,loss,val_acc,val_loss
base,0.771575,1.71937,0.714641,2.20225
simpler,0.657681,1.38791,0.644913,1.45527
dropout,0.5647,0.974106,0.614227,0.898497
dropout 0.2,0.625194,0.858744,0.639819,0.845117
batch norm,0.770882,0.768122,0.73088,0.930276
glorot,0.757019,0.736413,0.713525,0.919137
glorot_wo_bn,0.623706,0.900176,0.64571,0.896537
double,0.691916,0.759822,0.680806,0.804116
conv,0.952942,0.132164,0.754518,1.12405
conv1d,0.954887,0.128248,0.754804,1.0457


In [55]:
print(round(means.transpose() * 100, 2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &    acc &    loss &  val\_acc &  val\_loss \\
\midrule
base          &  77.16 &  171.94 &    71.46 &    220.22 \\
simpler       &  65.77 &  138.79 &    64.49 &    145.53 \\
dropout       &  56.47 &   97.41 &    61.42 &     89.85 \\
dropout 0.2   &  62.52 &   85.87 &    63.98 &     84.51 \\
batch norm    &  77.09 &   76.81 &    73.09 &     93.03 \\
glorot        &  75.70 &   73.64 &    71.35 &     91.91 \\
glorot\_wo\_bn  &  62.37 &   90.02 &    64.57 &     89.65 \\
double        &  69.19 &   75.98 &    68.08 &     80.41 \\
conv          &  95.29 &   13.22 &    75.45 &    112.40 \\
conv1d        &  95.49 &   12.82 &    75.48 &    104.57 \\
bidirectional &  86.82 &   36.99 &    77.08 &     73.27 \\
\bottomrule
\end{tabular}



The better results in terms of val_loss (more robust model) are the ones achieved by batch norm model, so we can assume this model will generalize better on predictions. The resuts we get on accuracy could depend highly on how the data is distributed, so if a model predicts a high rate os positive texts it could be getting higher accuracy due to there are a lot more positive texts in corpus

#### validation results

In [56]:
means_test = pd.concat([\
           lstm_base_evas.mean(),\
           lstm_simpler_evas.mean(),\
           lstm_dropout_evas.mean(),\
           lstm_dropout2_evas.mean(),\
           lstm_bn_evas.mean(),\
           lstm_glorot_evas.mean(),\
           lstm_glorot_wo_bn_evas.mean(),\
           lstm_double_evas.mean(),\
           lstm_conv_evas.mean(),\
           lstm_conv1d_evas.mean(),\
           lstm_bidirectional_evas.mean()
          ], axis=1)

In [57]:
means_test.columns = ['base', 'simpler', 'dropout', 'dropout 0.2', 'batch norm', 'glorot', 'glorot_wo_bn', 'double', 'conv', 'conv1d', 'bidirectional']

In [58]:
means_test.transpose().style.highlight_max()

,acc,f1,mse,recall
base,0.705921,0.488868,0.9332,0.490363
simpler,0.668729,0.461952,0.853826,0.468828
dropout,0.696407,0.461742,0.990818,0.453802
dropout 0.2,0.695476,0.461811,0.994544,0.447763
batch norm,0.75662,0.52068,0.749967,0.504872
glorot,0.756021,0.521104,0.752362,0.503337
glorot_wo_bn,0.726946,0.496822,0.868663,0.487015
double,0.754225,0.518277,0.759548,0.5037
conv,0.747638,0.522833,0.758949,0.533047
conv1d,0.758683,0.532122,0.710778,0.529146


In [59]:
means_test.transpose().to_latex()

'\\begin{tabular}{lrrrr}\n\\toprule\n{} &       acc &        f1 &       mse &    recall \\\\\n\\midrule\nbase          &  0.705921 &  0.488868 &  0.933200 &  0.490363 \\\\\nsimpler       &  0.668729 &  0.461952 &  0.853826 &  0.468828 \\\\\ndropout       &  0.696407 &  0.461742 &  0.990818 &  0.453802 \\\\\ndropout 0.2   &  0.695476 &  0.461811 &  0.994544 &  0.447763 \\\\\nbatch norm    &  0.756620 &  0.520680 &  0.749967 &  0.504872 \\\\\nglorot        &  0.756021 &  0.521104 &  0.752362 &  0.503337 \\\\\nglorot\\_wo\\_bn  &  0.726946 &  0.496822 &  0.868663 &  0.487015 \\\\\ndouble        &  0.754225 &  0.518277 &  0.759548 &  0.503700 \\\\\nconv          &  0.747638 &  0.522833 &  0.758949 &  0.533047 \\\\\nconv1d        &  0.758683 &  0.532122 &  0.710778 &  0.529146 \\\\\nbidirectional &  0.755356 &  0.539676 &  0.702728 &  0.545240 \\\\\n\\bottomrule\n\\end{tabular}\n'

#### Test results

In [14]:
lstm_base_pred['f_res'] = lstm_base_pred.apply(lambda x: np.argmax(x), axis=1)
lstm_simpler_preds['f_res'] = lstm_simpler_preds.apply(lambda x: np.argmax(x), axis=1)
lstm_dropout_preds['f_res'] = lstm_dropout_preds.apply(lambda x: np.argmax(x), axis=1)
lstm_dropout2_preds['f_res'] = lstm_dropout2_preds.apply(lambda x: np.argmax(x), axis=1)
lstm_bn_preds['f_res'] = lstm_bn_preds.apply(lambda x: np.argmax(x), axis=1)
lstm_glorot_preds['f_res'] = lstm_glorot_preds.apply(lambda x: np.argmax(x), axis=1)
lstm_glorot_wo_bn_preds['f_res'] = lstm_glorot_wo_bn_preds.apply(lambda x: np.argmax(x), axis=1)
lstm_double_preds['f_res'] = lstm_double_preds.apply(lambda x: np.argmax(x), axis=1)
lstm_conv_preds['f_res'] = lstm_conv_preds.apply(lambda x: np.argmax(x), axis=1)
lstm_conv1d_preds['f_res'] = lstm_conv1d_preds.apply(lambda x: np.argmax(x), axis=1)
lstm_bidirectional_preds['f_res'] = lstm_bidirectional_preds.apply(lambda x: np.argmax(x), axis=1)

lstm_base_pred_cine['f_res'] = lstm_base_pred_cine.apply(lambda x: np.argmax(x), axis=1)
lstm_simpler_preds_cine['f_res'] = lstm_simpler_preds_cine.apply(lambda x: np.argmax(x), axis=1)
lstm_dropout_preds_cine['f_res'] = lstm_dropout_preds_cine.apply(lambda x: np.argmax(x), axis=1)
lstm_dropout2_preds_cine['f_res'] = lstm_dropout2_preds_cine.apply(lambda x: np.argmax(x), axis=1)
lstm_bn_preds_cine['f_res'] = lstm_bn_preds_cine.apply(lambda x: np.argmax(x), axis=1)
lstm_glorot_preds_cine['f_res'] = lstm_glorot_preds_cine.apply(lambda x: np.argmax(x), axis=1)
lstm_glorot_wo_bn_preds_cine['f_res'] = lstm_glorot_wo_bn_preds_cine.apply(lambda x: np.argmax(x), axis=1)
lstm_double_preds_cine['f_res'] = lstm_double_preds_cine.apply(lambda x: np.argmax(x), axis=1)
lstm_conv_preds_cine['f_res'] = lstm_conv_preds_cine.apply(lambda x: np.argmax(x), axis=1)
lstm_conv1d_preds_cine['f_res'] = lstm_conv1d_preds_cine.apply(lambda x: np.argmax(x), axis=1)
lstm_bidirectional_preds_cine['f_res'] = lstm_bidirectional_preds_cine.apply(lambda x: np.argmax(x), axis=1)

/home/suampa/.local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning:


The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.



In [13]:
lstm_base_pred_cine

,0,1,2,3
0,2.179955e-05,4.214599e-01,1.512428e-01,4.272756e-01
1,5.113742e-08,1.093167e-03,1.508309e-03,9.973984e-01
2,5.996062e-05,3.760516e-01,9.684901e-02,5.270395e-01
3,2.583232e-34,1.000000e+00,9.412116e-09,2.485590e-09
4,3.446655e-05,1.676090e-01,8.080250e-02,7.515541e-01
5,2.323351e-07,7.788145e-01,1.004394e-01,1.207458e-01
6,2.101320e-10,9.323942e-01,3.119640e-02,3.640936e-02
7,8.968995e-13,1.993705e-05,9.339121e-06,9.999708e-01
8,3.598093e-10,9.235411e-01,3.164168e-02,4.481723e-02
9,6.875107e-05,3.822075e-01,1.022241e-01,5.154997e-01


In [15]:

preds = pd.concat([\
                   lstm_base_pred['f_res'],\
                   lstm_simpler_preds['f_res'],\
                   lstm_dropout_preds['f_res'],\
                   lstm_dropout2_preds['f_res'],\
                   lstm_bn_preds['f_res'],\
                   lstm_glorot_preds['f_res'],\
                   lstm_glorot_wo_bn_preds['f_res'],\
                   lstm_double_preds['f_res'],\
                   lstm_conv_preds['f_res'],\
                   lstm_conv1d_preds['f_res'],\
                   lstm_bidirectional_preds['f_res']\
                  ], axis=1)
preds_cine = pd.concat([\
                   lstm_base_pred_cine['f_res'],\
                   lstm_simpler_preds_cine['f_res'],\
                   lstm_dropout_preds_cine['f_res'],\
                   lstm_dropout2_preds_cine['f_res'],\
                   lstm_bn_preds_cine['f_res'],\
                   lstm_glorot_preds_cine['f_res'],\
                   lstm_glorot_wo_bn_preds_cine['f_res'],\
                   lstm_double_preds_cine['f_res'],\
                   lstm_conv_preds_cine['f_res'],\
                   lstm_conv1d_preds_cine['f_res'],\
                   lstm_bidirectional_preds_cine['f_res']\
                  ], axis=1)

In [61]:
preds = preds.applymap(lambda x: 1 if x >= 0.5 else 0)

In [16]:
preds.columns = ['base', 'simpler', 'dropout', 'dropout 0.2', 'batch norm', 'glorot', 'glorot_wo_bn', 'double', 'conv', 'conv1d', 'bidirectional']
preds_cine.columns = ['base', 'simpler', 'dropout', 'dropout 0.2', 'batch norm', 'glorot', 'glorot_wo_bn', 'double', 'conv', 'conv1d', 'bidirectional']

In [17]:
preds.shape

(1503, 11)

In [22]:
metrics = []
metrics_cine = []
for p in preds.columns:
    metrics.append(compute_metrics(preds[p], results_test['ls']['real'][0]))

for p in preds_cine.columns:
    metrics_cine.append(compute_metrics(preds_cine[p], results_test['ls']['cine_real'][0]))

In [23]:
pd.DataFrame(metrics, index = preds.columns).style.highlight_max(axis=0)

,acc,f1,mse,recall
base,0.742515,0.742515,0.806387,0.742515
simpler,0.752495,0.752495,0.766467,0.752495
dropout,0.713906,0.713906,0.920825,0.713906
dropout 0.2,0.75316,0.75316,0.763806,0.75316
batch norm,0.77179,0.77179,0.689288,0.77179
glorot,0.765802,0.765802,0.71324,0.765802
glorot_wo_bn,0.74185,0.74185,0.809049,0.74185
double,0.741184,0.741184,0.81171,0.741184
conv,0.732535,0.732535,0.776447,0.732535
conv1d,0.735196,0.735196,0.767798,0.735196


In [77]:
print(round(pd.DataFrame(metrics, index = preds.columns) * 100, 2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &    acc &     f1 &    mse &  recall \\
\midrule
base          &  74.25 &  74.25 &  80.64 &   74.25 \\
simpler       &  75.25 &  75.25 &  76.65 &   75.25 \\
dropout       &  71.39 &  71.39 &  92.08 &   71.39 \\
dropout 0.2   &  75.32 &  75.32 &  76.38 &   75.32 \\
batch norm    &  77.18 &  77.18 &  68.93 &   77.18 \\
glorot        &  76.58 &  76.58 &  71.32 &   76.58 \\
glorot\_wo\_bn  &  74.18 &  74.18 &  80.90 &   74.18 \\
double        &  74.12 &  74.12 &  81.17 &   74.12 \\
conv          &  73.25 &  73.25 &  77.64 &   73.25 \\
conv1d        &  73.52 &  73.52 &  76.78 &   73.52 \\
bidirectional &  75.58 &  75.58 &  71.52 &   75.58 \\
\bottomrule
\end{tabular}



In [24]:
print(round(pd.DataFrame(metrics_cine, index = preds.columns) * 100, 2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &    acc &     f1 &     mse &  recall \\
\midrule
base          &  34.02 &  34.02 &  141.52 &   34.02 \\
simpler       &  33.58 &  33.58 &  143.10 &   33.58 \\
dropout       &  30.90 &  30.90 &  154.00 &   30.90 \\
dropout 0.2   &  32.16 &  32.16 &  148.96 &   32.16 \\
batch norm    &  34.24 &  34.24 &  140.64 &   34.24 \\
glorot        &  22.78 &  22.78 &  186.48 &   22.78 \\
glorot\_wo\_bn  &  35.08 &  35.08 &  137.46 &   35.08 \\
double        &  31.48 &  31.48 &  151.96 &   31.48 \\
conv          &  35.74 &  35.74 &  133.32 &   35.74 \\
conv1d        &  29.40 &  29.40 &  160.06 &   29.40 \\
bidirectional &  34.92 &  34.92 &  137.86 &   34.92 \\
\bottomrule
\end{tabular}



In [ ]:
pd.concat([pd.DataFrame(results_test['lr']['real'][0]), preds ]).iplot(kind='histogram')